# Create a Database
Shamelessly plagerized from Ed Platt's [WP Efficiency Performance](https://github.com/elplatt/WP-Efficiency-Performance) repo.  (With permission.)

In [1]:
import configparser
import os
import sqlalchemy
#import schema  # This is a module you create w/ SQLAlchemy calls to create the DB tables

In [2]:
config = configparser.ConfigParser()
path = "../"
config_file = os.path.join(path, 'config.ini')
config.read(config_file)

['../config.ini']

From [this gist](https://gist.github.com/sprin/5846464):

In [6]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    Integer,
    String, 
    ForeignKey,
    DateTime,
    Boolean,
)
from sqlalchemy.pool import NullPool

In [7]:
# XXX: It is advised to use another user that can connect to a default database,
# and has CREATE DATABASE permissions, rather than use a superuser.
DB_CONFIG_DICT = {
        'user': config.get("db_config", "username"),
        'password': config.get("db_config", "password"),
        'host': config.get("db_config", "host"),
        'port': config.get("db_config", "port"),
    }

In [8]:
DB_CONN_FORMAT = "postgresql://{user}:{password}@{host}:{port}/{database}"

DB_CONN_URI_DEFAULT = (DB_CONN_FORMAT.format(
        database=config.get("db_config", "database"),
    **DB_CONFIG_DICT))

engine_default = create_engine(DB_CONN_URI_DEFAULT)

NEW_DB_NAME = config.get("db_config", "database")

DB_CONN_URI_NEW = (DB_CONN_FORMAT.format(
    database=NEW_DB_NAME,
    **DB_CONFIG_DICT))


conn = engine_default.connect()
meta = sqlalchemy.MetaData(engine_default)

slams = Table('slams', meta,
    Column('name', String, primary_key=True),
    Column('country', String)
)

results = Table('results', meta,
    Column('slam', String, ForeignKey('slams.name')),
    Column('year', Integer),
    Column('result', String)
)

meta.create_all(conn)
conn.close()

In [46]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
Base = declarative_base()

class Status(Base):
    '''Schema for bill status table '''
    __tablename__ = 'status'
    status_id = Column(Integer, primary_key=True)
    status_name = Column(String(200))

    
class VoteType(Base):
    '''Schema for vote type table'''
    __tablename__ = 'vote_type'
    vote_type_id = Column(Integer, primary_key=True)
    

class BillStatus(Base):
    '''Schema for bill status table'''
    __tablename__ = 'bill_status'
    bill_status_id = Column(Integer, primary_key=True)
    bill_id = Column(Integer)
    status_id = Column(Integer)
    date = Column(DateTime)
    
    
class Vote(Base):
    '''Schema for vote results table'''
    __tablename__ = 'vote'
    vote_id = Column(Integer, primary_key=True)
    bill_id = Column(Integer)
    repr_id = Column(Integer)
    voted = Column(Integer)
    vote_type_id = Column(Integer)
    
    
class Bill(Base):
    '''Schema for the bill contents table'''
    __tablename__  = 'bill'
    bill_id = Column(String(20), primary_key=True)
    display_name = Column(String(25))
    title = Column(String(1000))
    full_text_url = Column(String(140))
    sponsor_id = Column(Integer)
    committee_id = Column(Integer)
    current_status_id = Column(Integer)
    summary = Column(String(10000))
    
    
class Representative(Base):
    '''Schema for individual representatives info'''
    __tablename__ = 'representative'
    representative_id = Column(Integer, primary_key=True)
    chamber_name = Column(String(126))
    first_name = Column(String(60))
    last_name = Column(String(60))
    leadership_id = Column(Integer)
    dc_phone = Column(String(12))
    dc_address = Column(String(70))
    dc_office = Column(String(70))
    dc_fax = Column(String(12))
    district_office_phone = Column(String(12))
    district_office_address = Column(String(70))
    district_office = Column(String(70))
    district_office_fax = Column(String(70))
    contact_form = Column(String(100))
    website = Column(String(100))
    start_date = Column(DateTime)
    end_date = Column(DateTime)
    
class Leadership(Base):
    '''Schema for leadership positions table'''
    __tablename__ = 'leadership'
    leadership_id = Column(Integer, primary_key=True)
    chamber_name = Column(String(126))
    title = Column(String(126))
    
class Committee(Base):
    '''Schema for committee info table'''
    __tablename__ = 'committee'
    committee_id = Column(String(10), primary_key=True)
    display_name = Column(String(256))
    website = Column(String(256))
    obsolete = Column(Boolean)
    jurisdiction = Column(String(1000))
    chairman_repr_id = Column(Integer)
    chamber_name = Column(String(126))
    

class CommitteeMembership(Base):
    '''Schema for committee membership info'''
    __tablename__ = 'committee_membership'
    committee_membership_id = Column(Integer, primary_key=True)
    repr_id = Column(Integer)
    start_date = Column(DateTime)
    end_date = Column(DateTime)

In [47]:
Base.metadata.create_all(engine_default)

# Raw Data
## Where to get it

Run this at the command line to get bills from the 113th Congress.
```
rsync -avz --delete --delete-excluded --exclude **/text-versions/ \ govtrack.us::govtrackdata/congress/113/bills .
```

For more details, see [GovTrack's Bulk Data API documentation](https://www.govtrack.us/developers/data).


## What to track
Per the [Senate's website](http://www.senate.gov/legislative/bills.htm), there are four types of legislation.  Only 2 of them have the force of law.  (The other 2 are, presumably, to help our representatives... process their feelings?).

As such, we'll only track:

1. Bills
2. Joint Resolutions

And we'll ignore:

1. Concurrent Resolutions
2. Simple Resolution

So, `cd` into the `bills\ ` dir, and run the following at the command line to get rid of the stuff we don't need:
```
 rm -r hconres
 rm -r sconres
 rm -r  hres
 rm -r sres

```


In [88]:
!pwd

/Users/lruhlen/Projects/panopticon/notebooks


In [43]:
import json
import datetime as dt

In [4]:
base_data_dir = 'test_data/113_congress/bills'
senate_bills_dir = 's'
house_bills_dir = 'h'
senate_joint_res_dir = 'sjres'
house_joint_res_dir = 'hjres'

In [5]:
os.listdir(os.path.join(base_data_dir, senate_bills_dir))

['s1',
 's10',
 's100',
 's1000',
 's1001',
 's1002',
 's1003',
 's1004',
 's1005',
 's1006',
 's1007',
 's1008',
 's1009',
 's101',
 's1010',
 's1011',
 's1012',
 's1013',
 's1014',
 's1015',
 's1016',
 's1017',
 's1018',
 's1019',
 's102',
 's1020',
 's1021',
 's1022',
 's1023',
 's1024',
 's1025',
 's1026',
 's1027',
 's1028',
 's1029',
 's103',
 's1030',
 's1031',
 's1032',
 's1033',
 's1034',
 's1035',
 's1036',
 's1037',
 's1038',
 's1039',
 's104',
 's1040',
 's1041',
 's1042',
 's1043',
 's1044',
 's1045',
 's1046',
 's1047',
 's1048',
 's1049',
 's105',
 's1050',
 's1051',
 's1052',
 's1053',
 's1054',
 's1055',
 's1056',
 's1057',
 's1058',
 's1059',
 's106',
 's1060',
 's1061',
 's1062',
 's1063',
 's1064',
 's1065',
 's1066',
 's1067',
 's1068',
 's1069',
 's107',
 's1070',
 's1071',
 's1072',
 's1073',
 's1074',
 's1075',
 's1076',
 's1077',
 's1078',
 's1079',
 's108',
 's1080',
 's1081',
 's1082',
 's1083',
 's1084',
 's1085',
 's1086',
 's1087',
 's1088',
 's1089',
 's1

In [92]:
test_bill = 's100'
#os.listdir(os.path.join(base_data_dir, senate_bills_dir, test_bill))

data_file_name = 'data.json'
test_file = os.path.join(base_data_dir,
                         senate_bills_dir,
                         test_bill,
                         data_file_name)

bill_data = json.loads(open(test_file).read())
print('\n', bill_data.keys())
print('\n', 'official_title:\n', bill_data['official_title'])
print('\n', 'titles:\n', bill_data['titles'])
print('\n', 'short_title:\n', bill_data['short_title'])


 dict_keys(['bill_type', 'by_request', 'number', 'committees', 'summary', 'sponsor', 'amendments', 'url', 'status_at', 'actions', 'subjects_top_term', 'short_title', 'enacted_as', 'subjects', 'related_bills', 'cosponsors', 'congress', 'bill_id', 'history', 'titles', 'updated_at', 'introduced_at', 'popular_title', 'status', 'official_title'])

 official_title:
 A bill to amend the Financial Stability Act of 2010 to repeal certain designation authority of the Financial Stability Oversight Council, to repeal the Payment, Clearing, and Settlement Supervision Act of 2010, and for other purposes.

 titles:
 [{'is_for_portion': False, 'title': 'Terminating the Expansion of Too-Big-To-Fail Act of 2013', 'type': 'short', 'as': 'introduced'}, {'is_for_portion': False, 'title': 'A bill to amend the Financial Stability Act of 2010 to repeal certain designation authority of the Financial Stability Oversight Council, to repeal the Payment, Clearing, and Settlement Supervision Act of 2010, and for o

The `data.json` files contain the info we want.  

Here's the schema for the bills table, i.e. a list of what we need to extract:
```
class Bill(Base):
    '''Schema for the bill contents table'''
    __tablename__  = 'bill'
    bill_id = Column(String(20), primary_key=True)
    display_name = Column(String(25))
    title = Column(String(1000))
    full_text_url = Column(String(140))
    sponsor_id = Column(Integer)
    committee_id = Column(Integer)
    current_status_id = Column(Integer)

```

So, we need to get the bill's ID, long title (title), short title (display name), full text url, sponsor, committee(s), and current status.  

MAYBE ALSO MODIFY TABLE TO HAVE:
+ Bill's introduction date
+ Summary
+ List of subjects


See Tim's notes in my notebook about setting up a many-to-many table (with foreign keys!) for guidance on where to put the `bill_topics` values.

In [48]:
def get_bill_id(json_data=None):
    return json_data['bill_id']

def get_bill_display_name(json_data=None):
    return json_data['short_title']

def get_bill_title(json_data=None):
    return json_data['official_title']

def get_bill_summary(json_data=None):
    return json_data['summary']

def get_primary_bill_topic(json_data=None):
    return json_data['subjects_top_term']

def get_all_bill_topics(json_data=None):
    return json_data['subjects']

def get_is_bill_law(json_data=None):
    return json_data['history']['enacted']

def get_bill_committees(json_data=None):
    tmp = json_data['committees']
    tmp = set(x['committee_id'] for x in tmp)
    return list(tmp)

def get_bill_subcommittees(json_data=None):
    tmp = json_data['committees']
    tmp = set(x['subcommittee_id'] for x in tmp if 'subcommittee_id' in x)
    return list(tmp)

def get_primary_bill_sponsor(json_data=None):
    return json_data['sponsor']['thomas_id']

def get_bill_cosponsors(json_data=None):
    tmp = set(x['thomas_id'] for x in json_data['cosponsors'])
    return sorted(list(tmp))

def get_bill_congress_session(json_data=None):
    return json_data['congress']

def get_bill_introduction_date(json_data=None):
    str_date = json_data['introduced_at']
    return dt.datetime.strptime(str_date, '%Y-%m-%d')

In [26]:
def get_bill_status_date(json_data=None):
    return 'Placeholder for now'

def get_bill_status(json_data=None):
    return 'Placeholder for now'

def get_bill_info_url(json_data=None):
    return 'Placeholder for now'

def get_bill_actions(json_data=None):
    return 'Placeholder for now'

In [42]:
get_bill_cosponsors(bill_data)

['00066',
 '00116',
 '00136',
 '00172',
 '00174',
 '00179',
 '00250',
 '00326',
 '00501',
 '00604',
 '00791',
 '00802',
 '00859',
 '00949',
 '00952',
 '01010',
 '01036',
 '01247',
 '01332',
 '01381',
 '01384',
 '01409',
 '01424',
 '01507',
 '01531',
 '01541',
 '01546',
 '01558',
 '01567',
 '01595',
 '01647',
 '01694',
 '01701',
 '01820',
 '01823',
 '01826',
 '01828',
 '01829',
 '01837',
 '01844',
 '01850',
 '01863',
 '01866',
 '01897',
 '01898',
 '01900',
 '01901',
 '01902',
 '01937',
 '01965',
 '01969',
 '01983',
 '01984',
 '02075',
 '02076',
 '02173',
 '02174',
 '02176',
 '02182',
 '02185',
 '02189']

In [17]:
for item in get_bill_committees(json_data=bill_data):
    print(item.keys())

dict_keys(['committee', 'committee_id', 'activity'])
dict_keys(['committee', 'committee_id', 'activity'])
dict_keys(['subcommittee_id', 'committee', 'committee_id', 'activity', 'subcommittee'])


In [37]:
is_law = False
i=30

while not is_law:
    test_bill = ''.join(['s',str(i)])
    data_file_name = 'data.json'
    test_file = os.path.join(base_data_dir,
                             senate_bills_dir,
                             test_bill,
                             data_file_name)

    bill_data = json.loads(open(test_file).read())
    is_law = get_is_bill_law(bill_data)
    i += 1
    
print('\n\n')
for item in sorted(bill_data.keys()):
    print(item, '\t', bill_data[item], '\n')





actions 	 [{'text': 'Sponsor introductory remarks on measure.', 'acted_at': '2013-01-22', 'references': [{'reference': 'CR S157-158', 'type': None}], 'type': 'action'}, {'text': 'Introduced in the Senate. Read the first time. Placed on Senate Legislative Calendar under Read the First Time.', 'acted_at': '2013-01-22', 'references': [{'reference': 'CR S158-180', 'type': 'text of measure as introduced'}], 'type': 'action'}, {'acted_at': '2013-01-28', 'status': 'REPORTED', 'text': 'Read the second time. Placed on Senate Legislative Calendar under General Orders. Calendar No. 1.', 'number': '1', 'calendar': 'Senate Legislative', 'references': [], 'under': 'General Orders', 'type': 'calendar'}, {'text': 'Motion to proceed to consideration of measure made in Senate.', 'acted_at': '2013-01-31', 'references': [{'reference': 'CR S433', 'type': 'consideration'}], 'type': 'action'}, {'text': 'Motion to proceed to measure considered in Senate.', 'acted_at': '2013-02-04', 'references': [{'referen

In [11]:
subjects

NameError: name 'subjects' is not defined